In [ ]:
## Import packages
import os
import csv
import random
import math
import ntpath
import numpy as np
import pandas as pd
import time
from pathlib import Path
from scipy.io import wavfile

import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging

In [ ]:
import sys
sys.path.append('./vggish')

import vggish_input
import vggish_postprocess
import vggish_params
import vggish_slim

In [ ]:
# PARAMETERS
serval_data_folder = "../data/"

# we process train and eval files seperatly
# wav file that are in a '*_trn' folder land in the _trn folder
# wav file that are in a '*_eval' folder land in the _eval folder

In [ ]:
# delete flags if needed
delattr(flags.FLAGS, 'f')
delattr(flags.FLAGS, 'audio_embedding_feature_name')
delattr(flags.FLAGS, 'pca_params')
delattr(flags.FLAGS, 'checkpoint')


In [ ]:
flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string(
    'audio_embedding_feature_name', 'audio_embedding',
    'Path to the VGGish checkpoint file.')

## locatie (handmatig downloaden)
## https://github.com/tensorflow/models/tree/master/research/audioset/vggish
pca_params_vggish_model_folder = serval_data_folder + "vggish_model/models/"
print('Files in vggish model directory: (Expecting vggish_..., youtube_...)')
print(os.listdir(pca_params_vggish_model_folder))

flags.DEFINE_string(
    'pca_params', pca_params_vggish_model_folder + 'vggish_pca_params.npz',
    'Path to the VGGish PCA parameters file.')

flags.DEFINE_string(
    'checkpoint', pca_params_vggish_model_folder + 'vggish_model.ckpt',
    'Path to the VGGish checkpoint file.')

# flags.DEFINE_string(
#    'yt_checkpoint', 'models/youtube_model.ckpt',
#    'Path to the VGGish checkpoint file.')

In [ ]:
# TODO map to new directory structure in amsterdam_custom_samples
wav_samples_custom_path = serval_data_folder + "wav_samples_custom/"
wav_samples_custom_labels_csv = wav_samples_custom_path + "wav_samples_custom_labels.csv"
target_wav_samples_custom_all_enumerated_csv = wav_samples_custom_path + "wav_samples_custom_all_enumerated_and_labeled.csv"

# setup the locations for the tensorflow records
target_folder_tfrecords = serval_data_folder + "tf_records_samples/"
# folder extensions
trn_ext = '_trn'
eval_ext = '_eval'
target_folder_tfrecords_trn = target_folder_tfrecords + trn_ext + '/'
target_folder_tfrecords_eval = target_folder_tfrecords + eval_ext + '/'

In [ ]:
# read class labels
wav_samples_class_labels = pd.read_csv(wav_samples_custom_labels_csv, sep=";")
#print(wav_samples_class_labels)

In [ ]:
# Enumerate all the available wav files, label them and store a data-frame of the result
def collectAndLabelSamples(full_path_name, label, description, class_name):
    ## Read and rewrite all test files
    files = gfile.Glob(str(full_path_name + "*.wav"))
    
    df = pd.DataFrame(columns=['index', 'label', 'description', 'file_path', 'filename', 'class_name'])
    count = 1
    for file in files:
        df_row = pd.DataFrame({'index':count, 'label':[label], 'description':[description], 'file_path':[file], 'filename':[ntpath.basename(file)], 'class_name':[class_name]})
        df = df.append(df_row, ignore_index = True)
        count = count + 1
    return(df)

In [ ]:
## Enumerate all the available wav files, label them and store a data-frame of the result
# we have '_trn' and '_eval' folders now
# we do '_trn' first
folder_ext = eval_ext

df_all_wav_samples = pd.DataFrame(columns=['index', 'label', 'description', 'file_path', 'filename', 'class_name'])
for index, row in wav_samples_class_labels.iterrows():
    if row['enable'] == 1:
        print('Processing record ', index, '; label=', row.label, ', description=', row.description, ', path=', row.folder_name, sep='')
        df_row = collectAndLabelSamples(wav_samples_custom_path + row.folder_name + folder_ext + '/', row.label, row.description, row.folder_name )
        df_all_wav_samples = df_all_wav_samples.append(df_row, ignore_index = True)
    else:
        print('Skipping record ', index, '; label=', row.label, ', description=', row.description, ', path=', row.folder_name, sep='')

In [ ]:
## Write df_all_wav_samples to csv
print(df_all_wav_samples.head())
df_all_wav_samples.to_csv(target_wav_samples_custom_all_enumerated_csv, sep=';')

df_all_wav_samples.shape

In [ ]:
## Read all samples from disk in case you would like to change some by hand.
df_all_wav_samples = []
df_all_wav_samples = pd.read_csv(target_wav_samples_custom_all_enumerated_csv, sep=";")
print(df_all_wav_samples.head().loc[:,'file_path'][1])

In [ ]:
## Voor alle wav-files in een map, gebeurt het volgende:

## Eerst wordt het wav-file ingelezen en de bijbehorende label(s) opgezocht in een csv-bestand
## Als video-id wordt een willekeurig id'tje gegeven (in dit geval allemaal dezelfde)

## Dan parsen we wav-file naar embeddings: 
# (dit gebeurt door vggish_input.wavfile_to_example aan te roepen)
# Stap 1a: lezen van wav-file, input is array met samples die db aanduiden. Ook sample rate (per sec) wordt gelezen
# Stap 1b: Bij 2d array (stereo, ipv mono) bereken gemiddelde, daarna normaliseren (delen door 32.768)
# Stap 2: Bepaal examples in vorm [batch size, num frames, num bands].
    # Hierbij worden voor verschillende batches (omdat alles tegelijk niet in 1x in NN kan),
    # een log mel spectrogram gemaakt (in vorm [num_frames, num_bands])
# Stap 3: Bepaal features: nu wordt de embedding laag gemaakt (PCA-components, discreet maken etc)
    # Hiervoor worden model-parameters opgehaald die eerder zijn opgeslagen
    
## Daarna wordt een sequence example gemaakt (in getSequenceExample) en het als tf-records weggeschreven

In [ ]:
## Function that takes examples from wav-file as input and returns a sequence example

def getSequenceExample(examples_batch, labels, video_id=[b'-1LrH01Ei1w']):
    with tf.Graph().as_default(), tf.Session() as sess:

        # Prepare a postprocessor to munge the model embeddings.
        pproc = vggish_postprocess.Postprocessor(FLAGS.pca_params)
    
        # Define the model: load the checkpoint and locate input and output tensors
        # Input: [batch_size, num_frames, num_bands] 
        # where [num_frames, num_bands] represents log-mel-scale spectrogram
        # Output: embeddings
        vggish_slim.define_vggish_slim(training=False)
        vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)

        features_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_INPUT_TENSOR_NAME)
        embedding_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_OUTPUT_TENSOR_NAME)

        # Run inference and postprocessing.
        [embedding_batch] = sess.run([embedding_tensor],
                                     feed_dict={features_tensor: examples_batch})
        print(embedding_batch.shape)
        if(embedding_batch.shape!=10):
            print("*****************************************************************")

        postprocessed_batch = pproc.postprocess(embedding_batch)
        #print(postprocessed_batch)


        ## Maak labels en video-id voor in de example
        label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=labels))
        videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id))

        ## Maak sequence example
        seq_example = tf.train.SequenceExample(
            context = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat}),
            feature_lists = tf.train.FeatureLists(
                feature_list={
                    FLAGS.audio_embedding_feature_name:
                        tf.train.FeatureList(
                            feature=[
                                tf.train.Feature(
                                    bytes_list=tf.train.BytesList(
                                        value=[embedding.tobytes()]))
                                for embedding in postprocessed_batch
                            ]
                        )
                }
            )
        )
        
    return(seq_example)

In [ ]:
def encode_wav_files(wav_files, df_labels, output_directory, class_name):
    # setup a writer
    tf_record_writer = tf.python_io.TFRecordWriter(str(output_directory + class_name + '.tfrecord'))
    
    start = time.time()
    count = 0
    for file in wav_files:
        print("[INFO]: Count=", count, "/", len(wav_files), "; Time=", round((time.time() - start)), sep='')
        print("Filename: " + file, sep='')

        ## Fetch labels
        df_sample_labels = df_labels.loc[df_labels.filename == ntpath.basename(file)]
        if len(df_sample_labels) == 0:
            print('[ERROR]: Found sample without labels; filename=', file, sep='')
        else:
            ## .. encode - Part 1
            examples_batch = vggish_input.wavfile_to_examples(file)
            
            if(examples_batch.shape[0]>10):
                nr_sec = examples_batch.shape[0]
                print(nr_sec)
                start = int(math.floor((nr_sec-10)/2))
                print("start: " + str(start))
                end = int(nr_sec-math.ceil((nr_sec-10)/2))
                print("end: " + str(end))
                examples_batch = examples_batch[start:end, :, :]
                print(examples_batch.shape)
            
            ## .. encode - Part 2
            seq_example = getSequenceExample(examples_batch, df_sample_labels.label.unique())
            
            ## .. write to file
            tf_record_writer.write(seq_example.SerializeToString())
            
            ## .. done
            count = count + 1

    tf_record_writer.close()

In [ ]:
#  actual tf encoding
start = time.time()
for class_name in df_all_wav_samples.class_name.unique():
    df_class_wav_samples = df_all_wav_samples.loc[df_all_wav_samples.class_name == class_name]
    
    print('Processing: ', class_name, '; sample_count=', len(df_class_wav_samples), sep='')
    #print("[INFO]: Start time: ", round((time.time() - start)))
    # we add the folder extension here
    encode_wav_files(df_class_wav_samples.file_path, df_all_wav_samples, target_folder_tfrecords_eval , class_name)
    #print("[INFO]: End time: ", round((time.time() - start)))
    

In [ ]:
## Lees stuk voor stuk alle wav-files in
## Zoek het bijbehorende label op in een csv-bestand
## Bepaal de embeddings

# Prepare a record writer to store the postprocessed embeddings.
## Trainset
if(label == "gunshots"):
    train_tfrecord = str(tfrecord_path + 'gun_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'gun_test.tfrecord')
elif(label == "elephants_angela"):
    train_tfrecord = str(tfrecord_path + 'elephant_angela_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'elephant_angela_test.tfrecord')
elif(label == "background_angela"):
    train_tfrecord = str(tfrecord_path + 'background_angela_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'background_angela_test.tfrecord')
elif(label == "elephants_wenger"):
    train_tfrecord = str(tfrecord_path + 'elephant_wenger_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'elephant_wenger_test.tfrecord')
elif(label == "bush_wenger"):
    train_tfrecord = str(tfrecord_path + 'bush_wenger_train.tfrecord')
    test_tfrecord = str(tfrecord_path + 'bush_wenger_test.tfrecord')

    
    
train_writer = tf.python_io.TFRecordWriter(train_tfrecord)
test_writer = tf.python_io.TFRecordWriter(test_tfrecord)

## Read and rewrite all test files
files = gfile.Glob(str(wavfile_path + "*.wav"))

count = 1
for file in files:
    
    print("file nr: " + str(count))
    print("Filename: " + file)
    
    count += 1
    
    #print("Filename: " + str(file))
   
    ## Find labels belonging to wav-file
    if(label=="gunshots"):
        labels = getLabels(file)
        if(labels[0]==11):
            nr_9mm += 1
        elif(labels[0]==12):
            nr_556 += 1
    elif(label == "elephants_wenger"):
        labels = [13]
        elephants += 1
    elif(label == "bush_wenger"):
        labels = [14]
        bush += 1
    elif(label == "elephants_angela"):
        labels = [15]
        elephants_angela += 1
    elif(label == "background_angela"):
        labels = [16]
        elephants_angela += 1
    
    ## This function reads the wav file and converts the samples into np arrays of [batch size, num frames, num bands]
    #examples_batch = vggish_input.wavfile_to_examples(str(FLAGS.wavfile_path + wav_file))
    examples_batch = vggish_input.wavfile_to_examples(file)
    
    if(examples_batch.shape[0]>10):
        nr_sec = examples_batch.shape[0]
        print(nr_sec)
        start = int(math.floor((nr_sec-10)/2))
        print("start: " + str(start))
        end = int(nr_sec-math.ceil((nr_sec-10)/2))
        print("end: " + str(end))
        examples_batch = examples_batch[start:end, :, :]
        print(examples_batch.shape)
    
    #print("Examples shape: " + str(examples_batch.shape))
   
    seq_example = getSequenceExample(examples_batch, labels)
    
    rand = random.random()
    if rand <= pct_train:
        train_writer.write(seq_example.SerializeToString())
    else:
        test_writer.write(seq_example.SerializeToString())
    
train_writer.close()
test_writer.close()

In [ ]:
print("nr 9mm: " + str(nr_9mm))
print("nr 556: " + str(nr_556))

In [ ]:
print("nr elephants: " + str(elephants))

In [ ]:
print("nr bush: " + str(bush))

In [ ]:
print("nr gun google: " + str(gun_google))